In [9]:
#pip install yfinance

Цель работы: разработка программы, предлагающей поведенческую стратегию на основе индикаторов технического анализа.

Задачи работы:
* поиск данных;
* выбор индикатора технического анализа;
* реализация индикатора;
* тюнинг параметров индикатора.

В качестве исходных данных используем данные о стоимости акций компании Google за 2019-2022 гг.

В качестве индикатора для предложения поведенческой стратегии используем Stochastic RSI.

Stochastic RSI - индикатор, который является результатом применения стохастического осциллятора к индикатору RSI. Он колеблется в диапазоне от 0 до 1 и помогает определить перекупленные и перепроданные условия на рынке с большей точностью.

RSI рассчитывается следующим образом:
* для восходящего дня (дня, когда цена закрытия выше, чем предыдущая цена закрытия) положительные ценовые изменения Up = close_today - close_yesterday, D = 0
* для нисходящего дня (дня, когда цена закрытия ниже, чем предыдущая цена закрытия) положительные ценовые изменения Up = 0, D = close_yesterday - close_today.

Полученные значения сглаживаются с помощью экспоненциальной скользящей средней с заданным окном и находится их отношение:

RS = EMA(U, window) / EMA(D, window)

На основе RS рассчитывается RSI:

RSI = 100 - 100 / (1 + RS)

Стандартная формула Стохастического Осциллятора учитывает цену закрытия актива, а также его самые высокие и самые низкие точки в течение определенного периода. Однако когда формула используется для расчета Stochastic RSI, она непосредственно применяется к полученным значениям RSI (цены не учитываются):

Stochastic RSI = (current_RSI - min_RSI) / (max_RSI - min_RSI).

Stochastic RSI используется для различных целей:
* Определение перекупленных и перепроданных условий: если Stochastic RSI находится выше уровня 0.8, это указывает на перекупленные условия на рынке, и цена актива может измениться в обратном направлении. Если Stochastic RSI ниже уровня 0.2, это говорит о перепроданных условиях, и цена актива также может измениться в обратном направлении.
* Торговые сигналы: пересечение Stoch RSI определенных уровней может генерировать торговые сигналы. Если Stoch RSI пересекает уровень 0.2 снизу вверх, это сигнал к покупке. Если же Stoch RSI пересекает уровень 0.8 сверху вниз, это сигнал к продаже.
* Дивергенции: дивергенция между ценой актива и индикатором Stochastic RSI может указывать на возможные развороты тренда. Бычья дивергенция возникает, когда цена достигает новых минимумов, а Stochastic RSI не достигает новых минимумов. Это может сигнализировать о предстоящем развороте тренда вверх. Медвежья дивергенция возникает, когда цена достигает новых максимумов, а Stochastic RSI не достигает новых максимумов, что указывает на возможный разворот тренда вниз.




In [2]:
import yfinance as yf
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
df = yf.Ticker("GOOG").history(start='2019-01-01', end='2023-01-01')
df = df.reset_index()
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2019-01-02 00:00:00-05:00,50.828499,52.616001,50.785500,52.292500,30652000,0.0,0.0
1,2019-01-03 00:00:00-05:00,52.049999,52.848999,50.703499,50.803001,36822000,0.0,0.0
2,2019-01-04 00:00:00-05:00,51.629501,53.542000,51.370899,53.535500,41878000,0.0,0.0
3,2019-01-07 00:00:00-05:00,53.575001,53.700001,52.737999,53.419498,39638000,0.0,0.0
4,2019-01-08 00:00:00-05:00,53.805500,54.228001,53.026501,53.813999,35298000,0.0,0.0


In [4]:
def rsi(df, window):
  delta = df['Close'].diff()
  up, down = delta.copy(), delta.copy()
  up[up < 0]  = 0
  down[down > 0]  = 0
  average_gain = up.ewm(span=window, adjust=False, min_periods=window).mean()
  average_loss = abs(down.ewm(span=window, adjust=False, min_periods=window).mean())
  rs = average_gain / average_loss
  rsi = 100 - (100 / (1 + rs))
  return rsi

In [6]:
def stochastic_rsi(df, window):
    low_min  = df['rsi'].rolling(window=window).min()
    high_max = df['rsi'].rolling(window=window).max()
    stock_rsi = 100 * (df['rsi'] - low_min)/(high_max - low_min)
    return stock_rsi

Подберем лучшую величину окна для расчета значения индикатора:

In [7]:
best_rmse = 1000
best_window = 0
for window in range(2, 50):
  df['rsi'] = rsi(df, window)
  df['stochastic_rsi'] = stochastic_rsi(df, window=window)

  train = df[:850].query('stochastic_rsi == stochastic_rsi')
  test = df[850:]
  model = LinearRegression()
  model.fit(train['stochastic_rsi'].values.reshape(-1, 1), train.Close)
  rmse = mean_squared_error(test.Close, model.predict(test['stochastic_rsi'].values.reshape(-1, 1)), squared=False)

  if rmse < best_rmse:
    best_rmse = rmse
    best_window = window

df['rsi'] = rsi(df, best_window)
df['stochastic_rsi'] = stochastic_rsi(df, best_window)

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.Date.tail(180), y=df.Close.tail(180), name='Close price'))
fig.update_layout(title='Close price')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.Date.tail(180), y=df.stochastic_rsi.tail(180), name='Stochastic RSI'))
fig.add_shape(type="rect", xref="paper", yref="y", x0=0, y0=80, x1=1, y1=max(df.stochastic_rsi.tail(180)), fillcolor="red", opacity=0.3, layer="below", line_width=0)
fig.add_shape(type="rect", xref="paper", yref="y", x0=0, y0=20, x1=1, y1=min(df.stochastic_rsi.tail(180)), fillcolor="green", opacity=0.3, layer="below", line_width=0)
intersection_points = [i-1 for i in range(1, 180) if (df.stochastic_rsi.tail(180).values[i] < 81) & (df.stochastic_rsi.tail(180).values[i-1] > 80)]
fig.add_trace(go.Scatter(x=[df.Date.tail(180).iloc[i] for i in intersection_points], y=[80] * len(intersection_points), mode='markers', name='Sell', marker=dict(color='red', size=7)))
intersection_points = [i for i in range(1, 180) if (df.stochastic_rsi.tail(180).values[i] > 20) & (df.stochastic_rsi.tail(180).values[i-1] < 20)]
fig.add_trace(go.Scatter(x=[df.Date.tail(180).iloc[i] for i in intersection_points], y=[20] * len(intersection_points), mode='markers', name='Buy', marker=dict(color='green', size=7)))
fig.update_layout(title='Stochastic RSI')
fig.show()